In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
# For data visualization
import matplotlib.pyplot as plt
import seaborn as sns
# For displaying all of the columns in dataframes
pd.set_option('display.max_columns', None)
# For data modeling
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# For metrics and helpful functions
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import plot_tree
# For saving models
import pickle

In [ ]:
# #Preprocessing the images

# def pre_process(image):
#     # Resize image
#     resized_image = cv2.resize(image, (300, 300))

#     # Extract ROI (replace with your specific logic)
#     roi = resized_image[100:200, 150:250]

#     return roi

In [ ]:
def morphology_diff(contrast_green, clahe):

    #1st
    open1 = cv2.morphologyEx(contrast_green, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    close1 = cv2.morphologyEx(open1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    #2nd
    open2 = cv2.morphologyEx(close1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    close2 = cv2.morphologyEx(open2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    #3rd
    open3 = cv2.morphologyEx(close2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    close3 = cv2.morphologyEx(open3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)

    contrast_morph = cv2.subtract(close3, contrast_green)
    return clahe.apply(contrast_morph)

In [ ]:
#Noise Removal

def remove_noise(morph_image):
    ret, thr = cv2.threshold(morph_image, 15, 255, cv2.THRESH_BINARY)
    mask = np.ones(morph_image.shape[:2], dtype="uint8") * 255
    # Correct line:
    contours, hierarchy = cv2.findContours(thr.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) <= 200:
            cv2.drawContours(mask, [cnt], -1, 0, -1)
    im = cv2.bitwise_and(morph_image, morph_image, mask=mask)
    ret,fin_thr = cv2.threshold(im,15,255,cv2.THRESH_BINARY_INV)
    new_img = cv2.erode(fin_thr, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)
    return new_img

In [ ]:
#Blob Removal

def remove_blob(clear_image, org_image):
    fundus_eroded = cv2.bitwise_not(clear_image)
    xmask = np.ones(org_image.shape[:2], dtype="uint8") * 255
    # x1, xcontours, xhierarchy = cv2.findContours(fundus_eroded.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    xcontours, xhierarchy = cv2.findContours(fundus_eroded.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in xcontours:
        shape = "unidentified"
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)
        if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
            shape = "circle"
        else:
            shape = "veins"
        if(shape=="circle"):
            cv2.drawContours(xmask, [cnt], -1, 0, -1)

    finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)
    blood_vessels = cv2.bitwise_not(finimage)
    return blood_vessels

In [ ]:
#Vein detection

def detect_vessel(org_image):
    copy_org_image = org_image.copy()
    blue, green, red = cv2.split(org_image)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
    contrast_green = clahe.apply(green)
    morph_image = morphology_diff(contrast_green, clahe)
    clear_image = remove_noise(morph_image)
    fin_image = remove_blob(clear_image, org_image)
    i = 0
    j = 0
    for gr, fin in zip(green, fin_image):
        for g, f in zip(gr, fin):
            if(f == 0):
                green[i][j] = 255
            j = j + 1
        j = 0
        i = i + 1
    #return fin_image
    return fin_image, cv2.merge(( blue, green, red))


In [ ]:
if __name__ == "__main__":
    data_catalog = "data"
    raw_catalog = "raw_vessels"
    vessel_catalog = "vessels_images"
    files_names = [x for x in os.listdir(data_catalog) if os.path.isfile(os.path.join(data_catalog,x))]
    files_names.sort()

    for file_name in files_names:
        out_name = file_name.split('.')[0]
        org_image = cv2.imread(data_catalog + '/' + file_name)
        raw_vessel, vessel_image = detect_vessel(org_image)
        cv2.imwrite(raw_catalog + '/' + out_name + ".JPG", raw_vessel)
        cv2.imwrite(vessel_catalog + '/' + out_name + ".JPG", vessel_image)
